<a href="https://colab.research.google.com/github/PatrickStevensCGU/IST322_Summer2021_GroupProject/blob/main/IST322_GroupProject_YelpScrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
import time 
import random
import urllib

In [2]:
# this you need to do everytime if you do not choose "permantly mount" option. 
from google.colab import drive
drive.mount('/content/drive')

# Mount the local drive
%cd /content/drive/MyDrive/
if not os.path.exists('Mydata'):
  os.mkdir('Mydata')

Mounted at /content/drive
/content/drive/MyDrive


In [6]:
%cd /content/drive/MyDrive/Mydata
#df = pd.read_excel('SBRC_Doctor_Reviews.csv', names=['review_rating','date','review','restaurant','city','state','postal_code','restaurant_rating'], header=None, index_col=False)
original_doctor_df = pd.read_excel('SBRC_Doctor_Reviews.xlsx', index_col=False)
print(f'Total review count: {len(original_doctor_df)}')

/content/drive/MyDrive/Mydata
Total review count: 2607


In [7]:
original_doctor_df['doctorID'] = original_doctor_df.index
original_doctor_df = original_doctor_df[['doctorID'] + [col for col in original_doctor_df.columns if col != 'doctorID']]  # move the column to the front
original_doctor_df.head()

,doctorID,Business ID,Name,Business Category,URL,snippet text,Address,City,State code,zip code,clamed,closed,latitude,longitude,phone
0,0,rancho-wellness-rancho-cucamonga-7,Rancho Wellness,"Family Practicefamilydr, Weight Loss Centerswe...",https://www.yelp.com/biz/rancho-wellness-ranch...,First off let me say that it takes A LOT for m...,8231 Rochester AveRancho WellnessRancho Cucamo...,Rancho Cucamonga,CA,91730.0,1,0,34.103970,-117.544580,9.094838e+09
1,1,san-bernardino-medical-group-san-bernardino,San Bernardino Medical Group,"Family Practicefamilydr, Internal Medicineinte...",https://www.yelp.com/biz/san-bernardino-medica...,I have been coming to SBMG for over ten years ...,"1700 N Waterman AveSan Bernardino, CA 92404",San Bernardino,CA,92404.0,1,0,34.130330,-117.279740,9.098839e+09
2,2,st-bernardine-medical-center-baby-and-family-c...,St. Bernardine Medical Center Baby and Family ...,"Family Practicefamilydr, Pediatricianspediatri...",https://www.yelp.com/biz/st-bernardine-medical...,Disregard the fact that I'm a man. My wife was...,"403 E Highland AveSan Bernardino, CA 92404",San Bernardino,CA,92404.0,1,0,34.135776,-117.275612,9.098814e+09
3,3,beaver-medical-group-highland-2,Beaver Medical Group,"Family Practicefamilydr, Internal Medicineinte...",https://www.yelp.com/biz/beaver-medical-group-...,First time ever going to this particular Beave...,"7223 Church StSte CHighland, CA 92346",Highland,CA,92346.0,1,0,34.122251,-117.173181,9.098621e+09
4,4,ali-uzma-r-md-fontana-2,Ali Uzma R MD,"Pediatricianspediatricians,",https://www.yelp.com/biz/ali-uzma-r-md-fontana-2,I have been going to Dr. Ali for over 7 years ...,"Fontana, CA 92336",Fontana,CA,92336.0,1,0,34.156970,-117.482160,9.093553e+09


In [22]:
# I was assigned 870 - 1740
# base_doctor_url_list = original_doctor_df['URL'][800:999].tolist()
# base_doctor_url_list[0]

doctors_to_scrape = original_doctor_df[800:999]
doctors_to_scrape.head()

,doctorID,Business ID,Name,Business Category,URL,snippet text,Address,City,State code,zip code,clamed,closed,latitude,longitude,phone
800,800,daniel-villarosa-md-apple-valley,"Daniel Villarosa, MD","Fertilityfertility, Obstetricians & Gynecologi...",https://www.yelp.com/biz/daniel-villarosa-md-a...,I always have the best experience every appt. ...,"18064 Wika RdApple Valley, CA 92307",Apple Valley,CA,92307.0,0,0,34.540932,-117.269852,7.608136e+09
801,801,select-physical-therapy-ridgecrest,Select Physical Therapy,"Sports Medicinesportsmed, Physical Therapyphys...",https://www.yelp.com/biz/select-physical-thera...,"I recently moved to Ridgecrest, and was in see...","935 East Ridgcrest BlvdRidgecrest, CA 93555",Ridgecrest,CA,93555.0,1,0,35.622102,-117.650538,7.603711e+09
802,802,hawes-john-r-jr-do-apple-valley,Hawes John R Jr DO,"Doctorsphysicians,",https://www.yelp.com/biz/hawes-john-r-jr-do-ap...,I have been coming to Dr. Johns office for a w...,"16017 Tuscola RdSte BApple Valley, CA 92307",Apple Valley,CA,92307.0,0,0,34.542247,-117.268836,7.602422e+09
803,803,family-practice-associates-hesperia,Family Practice Associates,"Family Practicefamilydr,",https://www.yelp.com/biz/family-practice-assoc...,Sports Physical was cheap!!! Don't know why a...,"11919 Hesperia RdHesperia, CA 92345",Hesperia,CA,92345.0,1,0,34.467530,-117.291980,7.609481e+09
804,804,route-66-dot-medical-hesperia-2,Route 66 DOT Medical,"Doctorsphysicians,",https://www.yelp.com/biz/route-66-dot-medical-...,These people are great. This is my second year...,"8685 US Highway 395Unit BHesperia, CA 92344",Hesperia,CA,92344.0,0,0,34.408610,-117.399125,7.609801e+09


In [11]:
def random_sleep(min_seconds=60, max_seconds=90):
  sleep_seconds = random.randrange(min_seconds, max_seconds)
  print(f'Sleeping for {sleep_seconds} seconds...')
  time.sleep(sleep_seconds)

In [12]:
import requests
def url_to_soup(url, min_seconds=60, max_seconds=90):
  random_sleep(min_seconds, max_seconds)  # sleep at least a minute between any request
  print(f'Calling url: {url}')
  response = requests.get(url).text  # Make a GET request to the target URL to get the raw HTML data
  soup = BeautifulSoup(response,'html.parser')  # Use BeautifulSoup to parse HTML
  return soup

In [14]:
def add_doctor_without_review(doctors_without_reviews_df, doctorID):
  # rebuild the dataframe, but with the new doctor URL
  doc_list = doctors_without_reviews_df['doctorID'].tolist()
  doc_list.append(doctorID)  
  doctors_without_reviews_df = pd.DataFrame(doc_list, columns=['doctorID'])
  doctors_without_reviews_df.to_pickle('/content/drive/MyDrive/Mydata/doctors_without_reviews_df_PICKLE')
  print('************** NO REVIEWS FOUND **************')

In [15]:
def get_review_count(soup, review_count_class_ids):
  # review count is needed to know how many pages to scrape.  This ID can be one of multiple
  number_reviews = -1
  for review_count_class_id in review_count_class_ids:
    review_soup = soup.find(class_=review_count_class_id)
    if review_soup:      
      number_reviews = review_soup.text
      break;
  
  number_reviews_string_list = re.findall('\d+', number_reviews)
  if number_reviews_string_list:  # make sure the list isn't empty
    number_reviews = int(number_reviews_string_list[0])   # Use rex to extract the numbers from a string 
  else:
    return -1

  print(f'review count: {number_reviews}')
  return number_reviews

In [25]:
# Start with wherever we left off last
review_df = pd.read_pickle('/content/drive/MyDrive/Mydata/review_df_PICKLE')

# Persist the docs without reviews, so they can be skipped without the sleep delay just finding out again that there are no reviews.
doctors_without_reviews_df = pd.read_pickle('/content/drive/MyDrive/Mydata/doctors_without_reviews_df_PICKLE')

# Otherwise, use this if starting fresh
# columns = ['doctorID', 'base_doctor_url', 'review_username', 'review_rating', 'review_date', 'review']
# review_df = pd.DataFrame(columns=columns)
# columns = ['doctorID']
# doctors_without_reviews_df = pd.DataFrame(columns=columns)

In [26]:
print(len(review_df))
print(len(doctors_without_reviews_df))

0
0


In [ ]:
##### FOR TESTING START WITH 1 doctor. Comment this out when running for real.
#base_doctor_url_list = base_doctor_url_list[:2]
#base_doctor_url_list = base_doctor_url_list[:2]
#base_doctor_url_list = ['https://www.yelp.com/biz/a-doctors-weight-loss-clinic-moreno-valley-2']
#base_doctor_url_list = ['https://www.yelp.com/biz/a-doctors-weight-loss-clinic-moreno-valley-2?start=10']

#doctors_already_done = review_df['base_doctor_url'].unique()
#doctors_already_done = []

# Yelp HTML class IDs
review_count_class_ids = ['css-1h1j0y3', 'css-bq71j2']
review_class_id = 'review__373c0__3MsBX'  # was review__373c0__13kpL
star_class_id = 'i-stars__373c0___sZu0'  # was i-stars__373c0__1T6rz
username_class_id = 'css-166la90'
date_class_id = 'css-e81eai'
content_class_id = 'raw__373c0__tQAx6' # was raw__373c0__3rcx7

count_series = review_df['doctorID'].value_counts()

total_doctors_count = len(doctors_to_scrape)
for index, row in doctors_to_scrape.iterrows():

  # Values to use later
  original_base_doctor_url = row['URL'] # because the re-directs won't match the actual url used
  doctor_id = row['doctorID']

  debug_text = f'{index} of {index + total_doctors_count}: {original_base_doctor_url} {doctor_id}'
  # Check if this one was scraped before but had no reviews
  if doctor_id in doctors_without_reviews_df['doctorID'].values:
    print(f'{debug_text} - Previously searched - NO REVIEWS')
    continue

  # Check if this one was scraped before and is already in the data
  if doctor_id in count_series:
    previous_review_count = count_series[base_doctor_url]
    print(f'{debug_text} - Previously searched - {previous_review_count} reviews')
    continue

  # for the first URL, make sure to handle re-directs, which are common
  print(f'{debug_text} - New doc')
  base_doctor_url = original_base_doctor_url
  time.sleep (random.randint(30, 40))
  res = urllib.request.urlopen(original_base_doctor_url)
  final_url = res.geturl()
  if final_url != original_base_doctor_url:
    print(f'Handle redirect to: {final_url}')
    base_doctor_url = final_url

  # Start with souping the initial page with the URL from the input CSV
  first_soup = url_to_soup(base_doctor_url, min_seconds=120, max_seconds=130) # for the first of each doctor, wait at least 2 minutes 

  # From that first page, get the number of reviews
  number_reviews = get_review_count(first_soup, review_count_class_ids)

  # if there are no reviews, then we are done with this one
  if number_reviews <= 0:
    add_doctor_without_review(doctors_without_reviews_df, doctor_id)
    continue

  # Find all of the reviews using soup
  soup_findall_resultsets = []
  first_set_of_reviews = first_soup.findAll(class_=review_class_id)
  print(f'FIRST SET OF REVIEWS SCRAPED: Count of initial reviews in soup_findall_resultsets: {len(first_set_of_reviews)}')
  soup_findall_resultsets.append(first_set_of_reviews)

  # Create the rest of the URLs needed to get all reviews for this doctor
  subsequent_review_page_urls = []
  for i in range (0, number_reviews, 10):
    subsequent_review_page_urls.append(f'{base_doctor_url}?start={str(i)}')
  del subsequent_review_page_urls[0]  # Get rid of the first one, because adding start=0 is the same as the base url

  # Make the rest of the requests
  for url_to_request in subsequent_review_page_urls:
    next_soup = url_to_soup(url_to_request, min_seconds=60, max_seconds=90) # one minute to 1.5 minute wait
    new_resultset = next_soup.findAll(class_=review_class_id)
    print(f'NEXT SET OF REVIEWS SCRAPED: Count of reviews in new_resultset: {len(new_resultset)}')
    soup_findall_resultsets.append(new_resultset)

  # Combine the soup results into one ResultSet
  soup_resultset = soup_findall_resultsets[0]
  for soup_findall_resultset in soup_findall_resultsets[1:]:
    soup_resultset.extend(soup_findall_resultset)
    print(f'Updated count of results in soup_resultset: {len(soup_resultset)}')

  for review in soup_resultset:
    # extract attribute value using the key aria-label for star ratings. 
    review_rating_string = review.find (class_=star_class_id)['aria-label']
    # \d+ is a regular expression pattern to extract numbers from a string
    review_rating = float (re.findall('\d+', review_rating_string)[0])  

    review_username = review.find (class_=username_class_id).text # the user name for the review
    review_date = review.find (class_=date_class_id).text # the date of the review
    review_content = review.find(class_=content_class_id, attrs={'lang':'en'}).text  # the text of the review

    single_review = {'doctorID': doctor_id, 'base_doctor_url': original_base_doctor_url, 'review_username': review_username,
                     'review_rating': review_rating, 'review_date': review_date, 'review': review_content}
    review_df = review_df.append(single_review, ignore_index = True)
  
  review_df.to_pickle('/content/drive/MyDrive/Mydata/review_df_PICKLE')

print('DONE')

800 of 999: https://www.yelp.com/biz/daniel-villarosa-md-apple-valley 800 - New doc


In [26]:
review_df.to_csv('/content/drive/MyDrive/Mydata/review_df.csv')